# Notebook para generar las respuestas automáticas de un modelo de llm

---

En este caso se usa gpt-4 con una licencia de Azure

**Cargar las librerias**

In [ ]:
import os
from openai import AzureOpenAI
import pandas as pd
# print todays date and exact time
from datetime import datetime

**Cargar las licencias**

(En caso de usar solamente cargar la api de OpenAI se utilizan parámetros distintos)

In [ ]:
endpoint = ''
key=''
model_name='gpt-4'
client = AzureOpenAI(azure_endpoint=endpoint,api_version="2024-02-01",api_key=key)

### Loop para generar respuestas

In [ ]:
# Cargar el archivo de preguntas
questions = pd.read_csv("preguntas\questions3.csv" )
# read the csv file but remove the second column
questions = questions.drop(questions.columns[1], axis=1)
now = datetime.now()
lista_res= []
parametro ='Usando alrededor de 20 palabras, sin ser tan especifico: ' #Este string modifica el prompt al inicio
for i in range (questions.shape[0]):
    pregunta = questions["question"].loc[i]
    completion = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user",
        "content": parametro+pregunta,  # Modificiacion del prompt
    },], 
    )
    #print("# Pregunta ",i, pregunta)
    #print(completion.choices[0].message.content, "/n")
    lista_res.append(completion.choices[0].message.content)

# Ver respuestas en un dataframe
respuestas = {
    model_name:lista_res,
}
respuestas = pd.DataFrame.from_dict(respuestas)
respuestas

#### Guardar las respuestas con tiempo y notas de ejecución

In [ ]:
# transpose the dataframe
respuestas = respuestas.T
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
respuestas['tiempo'] = dt_string
respuestas['Notas']= 'P2GPT3_5_20palabras'
respuestas.reset_index(inplace=True)
# Change the name of a column
respuestas.rename(columns={"index": "model"}, inplace=True)
# Change the name of all the columns to string
respuestas.columns = respuestas.columns.astype(str)
# Cargar en caso de haber respuestas previas
if not os.path.exists("datos/r-gpt-4.csv"):
    respuestas.to_csv("r-gpt-4.csv", index=False)
else:
    df = pd.read_csv("r-gpt-4.csv")
    df.rename(columns={"Unnamed: 0": "model"}, inplace=True)
    df = pd.concat([df, respuestas], axis=0, ignore_index=True)
    # save the new data
    df.to_csv("datos/r-gpt-4.csv", index=False)